In [1]:
import math
import numpy as np
import wandb
from tqdm import tqdm
from torchvision import transforms
from src.utils import get_data, accuracy, save_Yte, augment_dataset, array_to_tensor
from src.kernels import *
from src.models import OneVsRestKRR
from src.hog import hog
from sklearn.model_selection import train_test_split 

In [2]:
Xtr, Xte, Ytr = get_data()

## Hyperparameter tuning

In [3]:
Xtr_, Xval_, Ytr_, Yval_ = train_test_split(Xtr, Ytr, test_size=0.1, random_state=123)
Xtr_tensor_ = array_to_tensor(Xtr_)
Xval_tensor_ = array_to_tensor(Xval_)

In [9]:
hog_fun = lambda img: hog(img,
    cells_per_block=3,
    normalization='L2-Hys',
)
Xtr_hog_ = np.array([hog_fun(img) for img in tqdm(Xtr_tensor_, desc="Computing HOG")])
Xval_hog_ = np.array([hog_fun(img) for img in Xval_tensor_])

Computing HOG: 100%|██████████| 4500/4500 [00:24<00:00, 184.18it/s]


In [11]:
krr = OneVsRestKRR(kernel=LogKernel(d=.5).kernel, lambd=1e-4)
krr.fit(Xtr_hog_, Ytr_)
print("avec HOG", accuracy(Yval_, krr.predict(Xval_hog_)))
# krr = OneVsRestKRR(kernel=WaveletKernel(a=1.).kernel, lambd=1e-4)
# krr.fit(Xtr_, Ytr_)
# print("sans HOG", accuracy(Yval_, krr.predict(Xval_)))

Fitting: 100%|██████████| 10/10 [00:16<00:00,  1.64s/it]


avec HOG 0.596


In [5]:
def get_sweep_id(parameters):
    """https://docs.wandb.ai/guides/sweeps/configuration"""
    sweep_config = {
        'method': "bayes",
        'metric': {
          'name': 'accuracy',
          'goal': 'maximize'
        },
        'parameters': parameters
    }
    sweep_id = wandb.sweep(sweep_config, project='project-name')

    return sweep_id

In [6]:
import math

def one_run():
    wandb.init(project='project-name', entity='entity', tags=["HOG+KRR"])
    # kernel =============================================================================
    kernel_name = wandb.config.kernel_name
    if kernel_name == "GaussianKernel":
        kernel = GaussianKernel(sigma=wandb.config.sigma).kernel
    elif kernel_name == "LogKernel":
        kernel = LogKernel(d=wandb.config.d).kernel
    elif kernel_name == "GHIKernel":
        kernel = GHIKernel(beta=wandb.config.beta).kernel
    elif kernel_name == "WaveletKernel":
        kernel = WaveletKernel(a=wandb.config.a).kernel
    
    # model ==============================================================================
    krr = OneVsRestKRR(
        kernel=kernel,
        lambd=wandb.config.lambd,
    )
    krr.fit(Xtr_hog_, Ytr_)
    wandb.run.summary["accuracy"] = accuracy(Yval_, krr.predict(Xval_hog_))

In [ ]:
parameters = {
    'lambd': {
        'distribution': 'log_uniform',
        'min': math.log(1e-5),
        'max': math.log(1e-3),
        # 'value': 1e-4
    },
    'cells_per_block': {
        'value': 3,
    },
    'normalization': {
        'value': 'L2-Hys',
    },
    # kernel -----------------------------------------------------------------------------
    'kernel_name': {
        'value': "LogKernel",
        # 'values': ["GaussianKernel", "WaveletKernel", "LogKernel", "GHIKernel"],
    },
    'd': {
        'values': [1/3, 1/2],
    },
    # 'beta': {
    #     'distribution': 'log_uniform',
    #     'min': math.log(1e-1),
    #     'max': math.log(1e1),
    # },
    # 'a': {
    #     'distribution': 'log_uniform',
    #     'min': math.log(1e-1),
    #     'max': math.log(1e1),
    # },
}
sweep_id = get_sweep_id(parameters)
%env WANDB_SILENT=True

wandb.agent(sweep_id, function=one_run)

## Predictions

In [8]:
transform = [
    transforms.RandomHorizontalFlip(p=.5),
    transforms.RandomAffine(degrees=(0,10), translate=(0.1,0.2), scale=(0.8,1.2)),
]
Xtr_augm, Ytr_augm = augment_dataset(Xtr, Ytr, repeat=2, transform=transform)
Xtr_tensor = array_to_tensor(Xtr_augm)
Xte_tensor = array_to_tensor(Xte)
hog_fun = lambda img: hog(img,
    cells_per_block=3,
    normalization='L2-Hys',
)
Xtr_hog = np.array([hog_fun(img) for img in tqdm(Xtr_tensor, desc="Computing HOG")])
Xte_hog = np.array([hog_fun(img) for img in Xte_tensor])
krr = OneVsRestKRR(kernel=LogKernel(d=2).kernel, lambd=1e-4)
krr.fit(Xtr_hog, Ytr_augm)
hog_fun = lambda img: hog(img,
    cells_per_block=3,
    normalization='L2-Hys',
)
print(f"accuracy on training set: {accuracy(Ytr_augm, krr.predict(Xtr_hog))}")
Yte = krr.predict(Xte_hog)
save_Yte(Yte, model_name="HOG_DA_Log_OneVsRestKRR_1")

Fitting: 100%|██████████| 10/10 [08:29<00:00, 50.99s/it]


accuracy on training set: 0.9311333333333334


## Log kernel data augmentation


In [4]:
from utils import show_images, tensor_to_array, augment_dataset


transform = [
    transforms.RandomHorizontalFlip(p=.6),
    transforms.RandomAffine(degrees=(0,10), translate=(0.1,0.2), scale=(0.8,1.2)),
]
Xtr_augm_, Ytr_augm_ = augment_dataset(Xtr_, Ytr_, repeat=2, transform=transform)
print(Xtr_augm_.shape)

Augmenting: 100%|██████████| 9000/9000 [00:08<00:00, 1053.15it/s]


(13500, 3072)


In [5]:
hog_fun = lambda img: hog(img,
    cells_per_block=3,
    normalization='L2-Hys',
)

Xtr_augm_tensor_ = array_to_tensor(Xtr_augm_)
# Xval_tensor_ = array_to_tensor(Xval_)
Xtr_augm_hog_ = np.array([hog_fun(img) for img in tqdm(Xtr_augm_tensor_, desc="Computing HOG")])
Xval_hog_ = np.array([hog_fun(img) for img in Xval_tensor_])


Computing HOG: 100%|██████████| 13500/13500 [01:32<00:00, 145.60it/s]


In [6]:

krr = OneVsRestKRR(kernel=LogKernel(d=2).kernel, lambd=1e-4)
krr.fit(Xtr_augm_hog_, Ytr_augm_)
print("avec data augm", accuracy(Yval_, krr.predict(Xval_hog_)))

Fitting: 100%|██████████| 10/10 [06:18<00:00, 37.86s/it]


avec data augm 0.626
